In [ ]:
!pip install -U gensim==4.0.0
!pip install python-Levenshtein
!pip install textattack

     |████████████████████████████████| 23.9MB 116kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149814 sha256=b86e2f5fa27871f862327ddd67684b485184210f6d7f26c5d26e46dc65e17bd2
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
     |████████████████████████████████| 358kB 18.8MB/s 
     |████████████████████████████████| 2.1MB 43.6MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 102kB 11.9MB/s 
     |████████████████████████████████| 225kB 53.0MB/s 
     |████████████████████████████████| 20.1MB 165kB/s 
     |████████████████████████████████| 778kB 39.8MB/s 
     |████████████████████████████████| 337kB 52.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
model1 = FastText.load("./drive/MyDrive/models/fasttext.model") # Loading our FastText model trained on text8 corpus

In [ ]:
model1.wv.most_similar('horse')

[('horsehair', 0.8985454440116882),
 ('horseshoe', 0.8598332405090332),
 ('horses', 0.854678213596344),
 ('horseshoes', 0.850151002407074),
 ('horseback', 0.8197551965713501),
 ('horseman', 0.8112313747406006),
 ('horseradish', 0.8012595772743225),
 ('horsecollar', 0.782683253288269),
 ('racehorse', 0.777208685874939),
 ('horsemen', 0.7719056606292725)]

### Defining our transformation:

In [ ]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return a word containing the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:07<00:00, 61.9MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpfpqamxsz.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-imdb")


model_wrapper = HuggingFaceModelWrapper(model, tokenizer) # Defines our target model

dataset = HuggingFaceDataset("imdb", None, "test") # Change the dataset


textattack: Loading datasets dataset imdb, split test.


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b. Subsequent calls will reuse this data.


## Defining our attack recipe:

In [ ]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(), StopwordModification()]
# # We'll use the Greedy search method
search_method = GreedyWordSwapWIR("weighted-saliency")
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


## Defining Faster Genetic Algorithm's recipe:

In [ ]:
!pip install torchfile

  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=8ff9907e84c9326e9a23b4b0371b7fd4f8e5f1c25a693241ced713ae460c0fb9
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


In [ ]:
from textattack.constraints.grammaticality.language_models import (
    LearningToWriteLanguageModel,
)
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import AlzantotGeneticAlgorithm
from textattack.shared.attack import Attack
from textattack.transformations import WordSwapEmbedding
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

transformation = WordSwapEmbedding(max_candidates=8)
        #
        # Don't modify the same word twice or stopwords
        #
constraints = [RepeatModification(), StopwordModification()]
        #
        # Maximum words perturbed percentage of 20%
        #
constraints.append(MaxWordsPerturbed(max_percent=0.2))
        #
        # Maximum word embedding euclidean distance of 0.5.
        #
constraints.append(WordEmbeddingDistance(max_mse_dist=0.5))
        #
        # Language Model
        #
        #
        #
constraints.append(
            LearningToWriteLanguageModel(
                window_size=6, max_log_prob_diff=5.0, compare_against_original=True
            )
        )
        # constraints.append(LearningToWriteLanguageModel(window_size=5))
        #
        # Goal is untargeted classification
        #
goal_function = UntargetedClassification(model_wrapper)
        #
        # Perform word substitution with a genetic algorithm.
        #
search_method = AlzantotGeneticAlgorithm(
            pop_size=60, max_iters=20, post_crossover_check=False
        )

# Now, let's make the attack from the 4 components:
attack1 = Attack(goal_function, constraints, transformation, search_method)

textattack: Downloading https://textattack.s3.amazonaws.com/constraints/grammaticality/language-models/learning-to-write.
100%|██████████| 1.04G/1.04G [00:17<00:00, 59.1MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpqwr7p7mn.zip to /root/.cache/textattack/constraints/grammaticality/language-models/learning-to-write.
textattack: Successfully saved constraints/grammaticality/language-models/learning-to-write to cache.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.01 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


# Attacking bert-base-uncased model:

In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-imdb")


model_wrapper = HuggingFaceModelWrapper(model, tokenizer) # Defines our target model

dataset = HuggingFaceDataset("imdb", None, "test") # Change the dataset


Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b)
textattack: Loading datasets dataset imdb, split test.


In [ ]:
data = []
for i in range(2000):
    data.append((list(dataset.__getitem__(i))[0]['text'],list(dataset.__getitem__(i))[1]))
for i in range(-2000,-1):
    data.append((list(dataset.__getitem__(i))[0]['text'],list(dataset.__getitem__(i))[1]))
data.sort(key = lambda x: len (x[0]))

## Attacking using our recipe:

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(data, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 0 / 1:   4%|▍         | 1/25 [00:00<00:05,  4.04it/s]

[Succeeded / Failed / Total] 0 / 1 / 2:   8%|▊         | 2/25 [00:01<00:13,  1.70it/s]

[Succeeded / Failed / Total] 0 / 2 / 3:  12%|█▏        | 3/25 [00:02<00:16,  1.33it/s]

[Succeeded / Failed / Total] 0 / 3 / 4:  16%|█▌        | 4/25 [00:03<00:17,  1.19it/s]

...

[Succeeded / Failed / Total] 14 / 9 / 25: 100%|██████████| 25/25 [00:29<00:00,  1.18s/it]

[Succeeded / Failed / Total] 15 / 9 / 26: : 26it [00:31,  1.20s/it]                      

[Succeeded / Failed / Total] 16 / 9 / 27: : 27it [00:32,  1.22s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 9      |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 33.33% |
| Attack success rate:          | 64.0%  |
| Average perturbed word %:     | 21.46% |
| Average num. words per input: | 30.67  |
| Avg num queries:              | 65.16  |
+-------------------------------+--------+


textattack: Attack time: 32.84748458862305s


## Attacking using Improved Genetic Algorithm's recipe:

In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack1.attack_dataset(data, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 0 / 1:   4%|▍         | 1/25 [00:00<00:00, 42.24it/s]

[Succeeded / Failed / Total] 0 / 1 / 2:   8%|▊         | 2/25 [01:57<22:32, 58.80s/it]

[Succeeded / Failed / Total] 0 / 2 / 3:  12%|█▏        | 3/25 [04:16<31:22, 85.56s/it]

[Succeeded / Failed / Total] 0 / 3 / 4:  16%|█▌        | 4/25 [07:19<38:28, 109.91s/it]

...

[Succeeded / Failed / Total] 8 / 15 / 25: 100%|██████████| 25/25 [57:24<00:00, 137.79s/it]

[Succeeded / Failed / Total] 8 / 16 / 26: : 26it [1:01:38, 142.25s/it]

[Succeeded / Failed / Total] 9 / 16 / 27: : 27it [1:02:08, 138.08s/it]

+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 9       |
| Number of failed attacks:     | 16      |
| Number of skipped attacks:    | 2       |
| Original accuracy:            | 92.59%  |
| Accuracy under attack:        | 59.26%  |
| Attack success rate:          | 36.0%   |
| Average perturbed word %:     | 15.85%  |
| Average num. words per input: | 30.67   |
| Avg num queries:              | 3292.84 |
+-------------------------------+---------+


textattack: Attack time: 3728.223778963089s
